In [1]:
import numpy as np
from numpy import pi
import qutip as qt
from qutip import hinton, identity, to_super, sigmaz, tensor, tensor_contract, sigmax, projection, jmat, basis, propagator, coefficient
import matplotlib.pyplot as plt
from functools import lru_cache

In [2]:
sqrt2 = np.sqrt(2)
isqrt2 = 1/np.sqrt(2)

In [3]:
# Hilbert space ordering is:
# NV - N15 - X - H1 

SzNV = tensor(jmat(1/2,'z'), identity([2,2,2]))
SxNV = tensor(jmat(1/2,'x'),identity([2,2,2]))
SyNV = tensor(jmat(1/2,'y'),identity([2,2,2]))

IzNV = tensor(identity([2]), jmat(1/2,'z'),identity([2,2]))
IxNV = tensor(identity([2]), jmat(1/2,'x'),identity([2,2]))
IyNV = tensor(identity([2]), jmat(1/2,'y'),identity([2,2]))

SzX = tensor(identity([2,2]), jmat(1/2,'z'), identity([2]))
SxX = tensor(identity([2,2]), jmat(1/2,'x'), identity([2]))
SyX = tensor(identity([2,2]), jmat(1/2,'y'), identity([2]))

IzX = tensor(identity([2,2,2]), jmat(1/2,'z'))
IxX = tensor(identity([2,2,2]), jmat(1/2,'x'))
IyX = tensor(identity([2,2,2]), jmat(1/2,'y'))

In [4]:
GHz=1e3
MHz=1
kHz=1e-3

ms=1e3
us=1
ns=1e-3

T = 1e4
G = 1

B = 360

gamma_e = -28024 * MHz / T
gamma_p = 42.577 * MHz / T
gamma_N = -4.316 * MHz / T

omegaNV =  2.87 * GHz + gamma_e * B
omegaX = gamma_e * B
omegaN15 = gamma_N * B
omegaP = gamma_p * B

Anv = 3.06 * MHz
D = 60 * kHz
Omega = 1 * MHz
Omega_e = 1 * MHz # formerly 1 MHz
Omega_h = 40*kHz
Omega_n = 20*kHz

T1e = 1*ms
T2e = 50*us
T1n = 10*ms
T2n = 500*us

T1e_rho = 100*us # this is emergent, no need to add directly

t_swap_nvx = 7.95*us
t_cnot_xn_n = 12.5*us
t_cnot_xn_e = 0.355*us
t_cnot_nvn_e = 0.355*us
t_cnot_nvn_n = 25*us

deg = np.pi/180

In [5]:
zdir_nv = (1/np.sqrt(3))*np.array([1,1,1])
theta_nv = np.arccos(1/np.sqrt(3))
phi_nv = 45*deg

def EulerMatrix(alpha, beta, gamma):
    return Rz(alpha) @ Ry(beta) @ Rz(gamma)

def Rz(alpha):
    return np.array(
        [
            [np.cos(alpha), -np.sin(alpha),0],
            [np.sin(alpha), np.cos(alpha),0],
            [0,0,1]
        ]
    )

def Ry(alpha):
    return np.array(
        [
            [np.cos(alpha), 0, np.sin(alpha)],
            [0,1, 0],
            [-np.sin(alpha),0, np.cos(alpha)]
        ]
    )

def Rx(alpha):
    return np.array(
        [
            [1,0,0],
            [0, np.cos(alpha), -np.sin(alpha),],
            [0, np.sin(alpha), np.cos(alpha),]
        ]
    )

In [6]:
# magnetic field alignment in the lab frame
theta = 0
phi = phi_nv

# some other relative angles 
alpha = 0*deg
beta = 90*deg

# principle components of uni-axial symm HF tensor
Ax_perp = 17.2 * MHz
Ax_para = 26 * MHz #29.4 * MHz

# direction of B field in the lab frame
Bdir = np.array([np.sin(theta)*np.sin(phi), np.sin(theta)*np.cos(phi), np.cos(theta)])

# Hyperfine tensor in the principle axis frame
Ax_principle = np.array([[Ax_perp, 0, 0], [0, Ax_perp, 0], [0, 0, Ax_para]])

RB = EulerMatrix(phi, theta, 0)
RA = EulerMatrix(beta, alpha, 0)

# NV-frame hyperfine tensor
Ax = RB @ RA.T @ Ax_principle @ RA @ RB.T



In [7]:
NvPi2 = (-1j*(np.pi/2)*SyNV).expm()
XPi2 = (-1j*(np.pi/2)*SyX).expm()

NV_Pi_2_super = qt.to_super(NvPi2)
X_Pi_2_super = qt.to_super(XPi2)

NV_Pi_super = NV_Pi_2_super**2

"""
Hartman-Hahn Hamiltonian for dipolar coupled electron spins
"""
H_spin_lock =  2*np.pi*(Omega * (SxNV + SxX)  + 2 * D * SzNV * SzX)


"""
Hyperfine Hamiltonians for X-H
HxHF_rE drives the electronic CNOT, RWA is taken judiciously
HxHF_rN drives the nuclear CNOT, includes terms that may be oscillating slowly enough to matter
"""
HxHF = (omegaX * SzX + omegaP * IzX + Ax[2,2] * (SzX * IzX) + Ax[2,0] * (SzX * IxX) + Ax[2,1] * (SzX * IyX))
energies = HxHF.eigenenergies()
omega_cnot21_MEAS = energies[3*4]-energies[0*4]
omega_cnot12_MEAS = energies[3*4]-energies[2*4]
cost = coefficient(lambda t: np.cos(omega_cnot12_MEAS*t))
sint = coefficient(lambda t: np.sin(omega_cnot12_MEAS*t))
HxHF_rE = 2*np.pi* ((omegaX + omega_cnot21_MEAS) * SzX + omegaP * IzX + Ax[2,2] * (SzX * IzX) + Ax[2,0] * (SzX * IxX) + Ax[2,1] * (SzX * IyX) + Omega_e*SxX)
HxHF_rN = 2*np.pi*( (omegaP+omega_cnot12_MEAS) * IzX + Ax[2,2] * (SzX * IzX) + Ax[2,0] * (SzX * (cost*IxX + sint*IyX)) + Ax[2,1] * (SzX * (cost*IyX - sint*IxX)) + Omega_h*IxX )

"""
Hyperfine Hamiltonians for NV-N
HnvHF_rE drives the electronic CNOT, RWA is taken judiciously
HnvHF_rN drives the nuclear CNOT, no perp components so no slow oscillators
"""
HnvHF =  (omegaNV * SzNV + omegaN15 * IzNV + Anv * (SzNV * IzNV))
energies_nv = HnvHF.eigenenergies()
omega_cnot21_MEAS_nv = energies_nv[1*4]-energies_nv[2*4]
omega_cnot12_MEAS_nv = energies_nv[1*4]-energies_nv[0*4]

HnvHF_rE = 2*np.pi* ((omegaNV + omega_cnot21_MEAS_nv) * SzNV + omegaN15 * IzNV + Anv * (SzNV * IzNV) + Omega_e*SxNV)
HnvHF_rN = 2*np.pi*( (omegaN15 + omega_cnot12_MEAS_nv) * IzNV + Anv * (SzNV * IzNV) + Omega_n*IxNV )

In [8]:
NV_c_ops = [1/(2*T1e) * SxNV, 1/(2*T1e) * SyNV, 1/(2*T2e) * SzNV]
X_c_ops = [1/(2*T1e) * SxX, 1/(2*T1e) * SyX, 1/(2*T2e) * SzX]

N15_c_ops = [1/(2*T1n) * IxNV, 1/(2*T1n) * IyNV, 1/(T2n) * IzNV]
H_c_ops = [1/(2*T1n) * IxX, 1/(2*T1n) * IyX, 1/(T2n) * IzX]

In [9]:
lindblad_ops = NV_c_ops + X_c_ops + N15_c_ops + H_c_ops

In [10]:
@lru_cache
def hartman_hahn_NV_X(t):
    # performs a hartman_hahn sequence on the electronic spins NV and X
    # Pi/2)_y - (simul spin lock along X) - Pi/2)_-y
    # variable time: iSWAP = 8.15us evolution time
    spin_lock = propagator(H_spin_lock, t,c_ops=lindblad_ops)
    simul_pulse = X_Pi_2_super * NV_Pi_2_super
    return simul_pulse.conj().trans() * spin_lock * simul_pulse

@lru_cache
def noise(t):
    # computes noise propegator for a time t while no 
    # other evolution is occuring, on all subspaces
    return propagator(identity([2,2,2,2]), t, c_ops=lindblad_ops)
    
@lru_cache
def partial_noise_nv(t):
    # get noise operator for only the NV-comb tooth
    # performs identity on other subspaces
    return propagator(identity([2,2,2,2]), t, c_ops=(NV_c_ops+ N15_c_ops))

@lru_cache
def partial_noise_x(t):
    # get noise operator for only the X-comb tooth
    # performs identity on other subspaces
    return propagator(identity([2,2,2,2]), t, c_ops=(X_c_ops+ H_c_ops))

@lru_cache
def cnot_x_nuc(t):
    # perform noisy selective nuclear transition on X-H
    # performs identity on other subspaces
    return propagator(HxHF_rN, t, c_ops=(X_c_ops+ H_c_ops), options={'nsteps':1e6})

@lru_cache
def cnot_x_e(t):
    # perform noisy selective electronic transition on X-H
    # performs identity on other subspaces
    return propagator(HxHF_rE, t, c_ops=(X_c_ops+ H_c_ops), options={'nsteps':1e6})

@lru_cache
def cnot_nv_nuc(t):
    # perform noisy selective nuclear transition on NV-N
    # performs identity on other subspaces
    return propagator(HnvHF_rN, t, c_ops=(NV_c_ops+ N15_c_ops), options={'nsteps':1e6})

@lru_cache
def cnot_nv_e(t):
    # perform noisy selective electronic transition on NV-N
    # performs identity on other subspaces
    return propagator(HnvHF_rE, t, c_ops=(NV_c_ops+ N15_c_ops), options={'nsteps':1e6})

@lru_cache
def partial_pol_state(pol):
    # returns a state partially polarize along Z
    # polarization must be a number between -1 and 1
    assert abs(pol) <= 1
    return (1+pol)/2*projection(2,0,0) + (1-pol)/2*projection(2,1,1)

def nv_reset(rho, pol):
    nrho = qt.vector_to_operator(noise(3*us)*qt.operator_to_vector(rho))
    prho = nrho.ptrace([2,3])
    return tensor(projection(2,0,0), partial_pol_state(pol), prho)

In [11]:
pol=0.8
rho0 = tensor(projection(2,0,0), partial_pol_state(pol),0.25*identity([2,2]))
rho0

Quantum object: dims=[[2, 2, 2, 2], [2, 2, 2, 2]], shape=(16, 16), type='oper', dtype=CSR, isherm=True
Qobj data =
[[0.225 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.225 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.225 0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.225 0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.025 0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.025 0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.025 0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.025 0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.    0.    0.    0.   ]
 [0

In [12]:
def NV_X_entanglement(rho):
    rho1 = hartman_hahn_NV_X(t_swap_nvx) * qt.operator_to_vector(rho)
    rho2 = nv_reset(qt.vector_to_operator(rho1),pol)
    return qt.vector_to_operator(hartman_hahn_NV_X(t_swap_nvx/2) * NV_Pi_super * qt.operator_to_vector(rho2))

In [13]:
rhoEnt = NV_X_entanglement(rho0)

In [14]:
qt.concurrence(rhoEnt.ptrace([0,2]))

0.9932762640706683

In [15]:
rhoEnt.ptrace([0,2])

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 9.93748042e-05-7.96058576e-17j -1.02215683e-16-1.75134861e-17j
   1.43786235e-16+9.13976393e-17j -1.80850812e-06+1.22406639e-05j]
 [ 1.78745170e-16-4.42178243e-16j  4.62492699e-01+2.88765173e-16j
  -8.49071190e-09-4.97178714e-01j -1.11835728e-16+7.69095185e-17j]
 [ 2.34445212e-16-8.14140751e-17j -8.49071234e-09+4.97178714e-01j
   5.34467250e-01+1.00787031e-16j  7.16030301e-17-3.12153293e-17j]
 [-1.80850812e-06-1.22406639e-05j  2.67139448e-16+3.18929187e-17j
  -2.40127206e-17-2.19793112e-16j  2.94067612e-03-1.82454688e-16j]]

In [16]:
swap_x = cnot_x_e(t_cnot_xn_e) * cnot_x_nuc(t_cnot_xn_n) * cnot_x_e(t_cnot_xn_e)
t_swap_x = 2*t_cnot_xn_e + t_cnot_xn_n

In [17]:
swap_nv = cnot_nv_e(t_cnot_nvn_e) * cnot_nv_nuc(t_cnot_nvn_n) * cnot_nv_e(t_cnot_nvn_e)
t_swap_nv = 2*t_cnot_nvn_e + t_cnot_nvn_n

In [18]:
t_swap_nv-t_swap_x

12.5

In [19]:
noise_x = partial_noise_x(t_swap_nv-t_swap_x)

In [20]:
rhoNucEnt = qt.vector_to_operator(noise_x*swap_x*swap_nv*qt.operator_to_vector(rhoEnt))

In [21]:
rhoNucEnt.ptrace([1,3])

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.07060688+3.15613266e-17j  0.01252944-9.63322530e-02j
  -0.11210465-1.91676165e-02j  0.02620558-3.19914019e-02j]
 [ 0.01252944+9.63322530e-02j  0.39464657+1.53651067e-16j
  -0.00239705-2.83612256e-01j  0.10267375+2.12996463e-02j]
 [-0.11210465+1.91676165e-02j -0.00239705+2.83612256e-01j
   0.45724568+3.65024827e-17j -0.017255  +1.06934081e-01j]
 [ 0.02620558+3.19914019e-02j  0.10267375-2.12996463e-02j
  -0.017255  -1.06934081e-01j  0.07750087-9.89750625e-17j]]

In [22]:
qt.concurrence(rhoNucEnt.ptrace([1,3]))

0.5546329695904562

In [45]:
swap_back_rho = noise_x*swap_x*swap_nv*qt.operator_to_vector(rhoNucEnt)

In [43]:
qt.vector_to_operator(swap_back_rho.ptrace([0,2]))

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[ 0.16653835-1.67735721e-16j -0.00744386+8.71653776e-02j
   0.04523242+1.21968164e-01j  0.06568734-1.58472134e-03j]
 [-0.00744386-8.71653776e-02j  0.34359613+3.33789903e-16j
   0.01248708-8.44183870e-02j -0.02541243+1.54007940e-03j]
 [ 0.04523242-1.21968164e-01j  0.01248708+8.44183870e-02j
   0.35250031+1.91969853e-16j  0.01378565-9.96829409e-02j]
 [ 0.06568734+1.58472134e-03j -0.02541243-1.54007940e-03j
   0.01378565+9.96829409e-02j  0.13736521-2.42340877e-16j]]

In [44]:
qt.concurrence(qt.vector_to_operator(swap_back_rho.ptrace([0,2])))

0.04770395350024491

In [33]:
qt.vector_to_operator(swap_back_rho.ptrace([0]))

Quantum object: dims=[[2], [2]], shape=(2, 2), type='oper', dtype=Dense, isherm=True
Qobj data =
[[0.51013448+1.63875538e-16j 0.01981999+1.23508243e-01j]
 [0.01981999-1.23508243e-01j 0.48986552-4.92901003e-17j]]

In [39]:
nv_recurrence = qt.vector_to_operator(swap_back_rho)
qt.expect(nv_recurrence,SzNV)

0.010134483706843174